In [ ]:
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3.common.monitor import Monitor
from wandb.integration.sb3 import WandbCallback
from sb3_contrib.ppo_mask import MaskablePPO
from env import MPSPEnv, MPSPObservationWrapper
import wandb
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'sb3.ipynb'
wandb.login()

In [ ]:
config = {
    # Environment
    'ROWS': 5,
    'COLUMNS': 5,
    'N_PORTS': 7,
    # Training
    'total_timesteps': 30000,
}

In [ ]:
run = wandb.init(
    project="Q-learning",
    entity="rl-msps",
    sync_tensorboard=True,
    name=f"Cartpole-SB3",
    config=config,
    tags=["test"]
)

In [ ]:
def mask_fn(env):
    return env.get_action_mask()


env = MPSPEnv(
    config['ROWS'],
    config['COLUMNS'],
    config['N_PORTS']
)
env = MPSPObservationWrapper(env)
env = Monitor(env)
env = ActionMasker(env, mask_fn)

In [ ]:
model = MaskablePPO(
    MaskableActorCriticPolicy,
    env,
    verbose=1,
    tensorboard_log=f"runs/{run.id}"
)

In [ ]:
model.learn(
    total_timesteps=config['total_timesteps'],
    callback=WandbCallback(
        model_save_path=f"models/{run.id}",
    )
)

In [ ]:
run.finish()